In [1]:
import pickle
import h5py
import os
import torch
import random
import torch.utils.data as data
import torch.nn as nn
from torch.autograd import Variable
import time
import numpy


/home/swjung/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Set HyperParameter
# learning_rate = 0.00001
learning_rate = 0.00001
gradient_direction = 0.9
batch_size = 310
epochs = 250
test_range = 32
total_process_count = test_range * epochs

TEST_TYPE = 0

if TEST_TYPE == 0:
    _type = 'Valence'
elif TEST_TYPE == 1:
    _type = 'Arousal'
else:
    raise ValueError('TEST_TYPE must be 1 or 2')

In [3]:
class DEAP_DataSet(data.Dataset):
    
    VALID_SPLIT = ('train', 'validation', 'test')
    DATASET = 'data/'
    
    def __init__(self, test_target, test_type=0, split='train', data_dir='processed_data.pkl', label_dir='y_train.pkl'):
        super(DEAP_DataSet, self).__init__()
        
        if split not in self.VALID_SPLIT:
            raise ValueError('Unknown split {:s}'.format(split))
        if not os.path.exists(self.DATASET + data_dir):
            raise ValueError('{:s} does not exist'.format(data_dir))
        if not os.path.exists(self.DATASET + label_dir):
            raise ValueError('{:s} does not exist'.format(label_dir))
            
        data_path = '{}/{}'.format(self.DATASET, data_dir)
        label_path = '{}/{}'.format(self.DATASET, label_dir)
        
        self.split = split
        self.data = pickle.load(open(data_path, 'rb'))
        self.labels = pickle.load(open(label_path, 'rb'))
        
        self.test_target = test_target
        self.test_type = test_type
        self.target_data = []
        
        # create train data set
        if self.split == self.VALID_SPLIT[0]:
            counter = 0
            for pp, pp_data in enumerate(self.data):
                if self.test_target is not pp:
                    for exp, data in enumerate(pp_data):
                        counter += 1
                        temp_label = self.labels[pp][exp][self.test_type]
                        temp_data = data.flatten()
                        self.target_data.append((temp_label, temp_data))
            
        # craete test data set
        if self.split == self.VALID_SPLIT[2]:
            counter = 0
            for pp, pp_data in enumerate(self.data):
                if self.test_target == pp:
                    for exp, data in enumerate(pp_data):
                        counter += 1
                        temp_label = self.labels[pp][exp][self.test_type]
                        temp_data = data.flatten()
                        self.target_data.append((temp_label, temp_data))
            
    def __getitem__(self, index):
        return self.target_data[index]
    
    def __len__(self):
        return len(self.target_data)

In [4]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(4040, 5000),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(5000, 500),
            nn.ReLU(),
            nn.Dropout(0.50),
            nn.Linear(500, 1000),
            nn.ReLU(),
            nn.Dropout(0.50),
            nn.Linear(1000, 2),
#             nn.Dropout(0.50),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        out = self.layer(x)
        return out

In [5]:
def run_train(test_num, test_type):
    train_model = DNN().double().cuda()
    
    DEAP_train = DEAP_DataSet(test_num, test_type=test_type, split='train')
    train_loader = torch.utils.data.DataLoader(DEAP_train, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)
    
    optimizer = torch.optim.RMSprop(train_model.parameters(), lr=learning_rate, alpha=gradient_direction)
#     optimizer = torch.optim.SGD(train_model.parameters(), lr=learning_rate, momentum=0.9)
    loss_func = nn.CrossEntropyLoss().double()
    
    for epoch in range(epochs):
        count = 0
        for label, data in train_loader:
            x = Variable(data).double().cuda()
            y_ = Variable(label).type(torch.LongTensor).cuda()

            optimizer.zero_grad()
            output = train_model.forward(x)
            loss = loss_func(output, y_).double()
#             print(loss)
            loss.backward()
            optimizer.step()

            count += 1
    return train_model

In [6]:
def run_validation(model, test_num, test_type):
    DEAP_test = DEAP_DataSet(test_num, test_type=test_type, split='test')
    test_loader = torch.utils.data.DataLoader(DEAP_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)
    
    total = 0
    correct = 0
    
    for label, data in test_loader:
        x = Variable(data, requires_grad=False).double().cuda()
        y_ = Variable(label).type(torch.LongTensor).cuda()
        
        with torch.no_grad():
            output = model.forward(x)
            _, output_index = torch.max(output, 1)

            total += label.size(0)
            correct += (output_index == y_).sum().float()
    return total, correct

In [7]:
def get_min_sec(t):
    _min = int(t / 60)
    _sec = int(t) % 60
    
    return _min, _sec

In [8]:
total = 0
correct = 0
for test_num in range(test_range):
    
    start_time = time.time()
    print('# > [{}/{}] train start.'.format(test_num + 1, test_range))
    model = run_train(test_num, TEST_TYPE)
    print('# > [{}/{}] train end.'.format(test_num + 1, test_range))
    
    temp_total = 0
    temp_correct = 0
    for c in range(30):
        sub_total, sub_correct = run_validation(model, test_num, TEST_TYPE)
        temp_total += sub_total
        temp_correct += sub_correct
        
    current_process_count = (test_num + 1) * epochs
    end_time = time.time()
    run_time = (end_time - start_time)
    remain_process = test_range - 1 - test_num
    remain_time = run_time * remain_process
    
    _min, _sec = get_min_sec(remain_time)
    
    print('# > [{}/{}] correct : {}% ({} / {}) -> {}%, remain time : {}mins {}secs'.format(test_num + 1, test_range, 100 * temp_correct / temp_total, temp_correct, temp_total, 100 * current_process_count / total_process_count, _min, _sec))
    
    total += temp_total
    correct += temp_correct
    print('# > total :  correct : {}% ({} / {})'.format(100 * correct / total, correct, total))

# > [1/32] train start.
tensor(0.8455, dtype=torch.float64, device='cuda:0')
tensor(0.8133, dtype=torch.float64, device='cuda:0')


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
print('# > train and validation end')
print('# > final report')
print('type : {}, result : {}% ({} / {})'.format(_type, 100 * correct / total, correct, total))

In [ ]:

TEST_TYPE = 1

if TEST_TYPE == 0:
    _type = 'Valence'
elif TEST_TYPE == 1:
    _type = 'Arousal'
else:
    raise ValueError('TEST_TYPE must be 1 or 2')

In [ ]:
total = 0
correct = 0
for test_num in range(test_range):
    
    start_time = time.time()
    print('# > [{}/{}] train start.'.format(test_num + 1, test_range))
    model = run_train(test_num, TEST_TYPE)
    print('# > [{}/{}] train end.'.format(test_num + 1, test_range))
    
    temp_total = 0
    temp_correct = 0
    for c in range(30):
        sub_total, sub_correct = run_validation(model, test_num, TEST_TYPE)
        temp_total += sub_total
        temp_correct += sub_correct
        
    current_process_count = (test_num + 1) * epochs
    end_time = time.time()
    run_time = (end_time - start_time)
    remain_process = test_range - 1 - test_num
    remain_time = run_time * remain_process
    
    _min, _sec = get_min_sec(remain_time)
    
    print('# > [{}/{}] correct : {}% ({} / {}) -> {}%, remain time : {}mins {}secs'.format(test_num + 1, test_range, 100 * temp_correct / temp_total, temp_correct, temp_total, 100 * current_process_count / total_process_count, _min, _sec))
    
    total += temp_total
    correct += temp_correct
    print('# > total :  correct : {}% ({} / {})'.format(100 * correct / total, correct, total))

In [ ]:
print('# > train and validation end')
print('# > final report')
print('type : {}, result : {}% ({} / {})'.format(_type, 100 * correct / total, correct, total))